In [1]:
import os
import shutil
import numpy as np
import pandas as pd
import pickle as pkl
import tifffile as tif
from multiprocessing import Pool

os.chdir("../../")

In [4]:
i = 0
class data_preprocessing:
    
    paths = []
    
    def __init__(self):
        with open("Data/hierarchy_data.pkl","rb") as f:
            hd = pkl.load(f)
        with open("Data/class_encoding.pkl","rb") as f:
            self.classes = pkl.load(f)
        with open("Data/order_encoding.pkl","rb") as f:
            self.orders = pkl.load(f)
        with open("Data/family_encoding.pkl","rb") as f:
            self.families = pkl.load(f)
        with open("Data/genus_encoding.pkl","rb") as f:
            self.genuses = pkl.load(f)
        with open("Data/specie_encoding.pkl","rb") as f:
            self.species = pkl.load(f)
    
        try:
            shutil.rmtree("Data/Hierarchial Data")
            os.mkdir("Data/Hierarchial Data")
            
        except:
            os.mkdir("Data/Hierarchial Data")
            
        os.mkdir("Data/Hierarchial Data/train")
        os.mkdir("Data/Hierarchial Data/test")
        
        for cls in hd.keys():
            os.mkdir("Data/Hierarchial Data/train/"+str(self.classes[cls]))
            os.mkdir("Data/Hierarchial Data/test/"+str(self.classes[cls]))
            for order in hd[cls].keys():
                os.mkdir("Data/Hierarchial Data/train/"+str(self.classes[cls])+"/"+str(self.orders[order]))
                os.mkdir("Data/Hierarchial Data/test/"+str(self.classes[cls])+"/"+str(self.orders[order]))
                for fam in hd[cls][order].keys():
                    os.mkdir("Data/Hierarchial Data/train/"+str(self.classes[cls])+"/"+str(self.orders[order])+"/"+str(self.families[fam]))
                    os.mkdir("Data/Hierarchial Data/test/"+str(self.classes[cls])+"/"+str(self.orders[order])+"/"+str(self.families[fam]))
                    for gen in hd[cls][order][fam].keys():
                        os.mkdir("Data/Hierarchial Data/train/"+str(self.classes[cls])+"/"+str(self.orders[order])+"/"+str(self.families[fam])+"/"+str(self.genuses[gen]))
                        os.mkdir("Data/Hierarchial Data/test/"+str(self.classes[cls])+"/"+str(self.orders[order])+"/"+str(self.families[fam])+"/"+str(self.genuses[gen]))
                        for spe in hd[cls][order][fam][gen]:
                            try:
                                os.mkdir("Data/Hierarchial Data/train/"+str(self.classes[cls])+"/"+str(self.orders[order])+"/"+str(self.families[fam])+"/"+str(self.genuses[gen])+"/"+str(spe))
                                os.mkdir("Data/Hierarchial Data/test/"+str(self.classes[cls])+"/"+str(self.orders[order])+"/"+str(self.families[fam])+"/"+str(self.genuses[gen])+"/"+str(spe))
                            except:
                                pass

        

    def train_test_src_dest_map(self, train_size=0.85):
        df = pd.read_csv("occurrences_train.csv", low_memory=False)
        l = list(range(df.shape[0]))
        np.random.shuffle(l)
        split = int(train_size *len(l)) 
        for i in l[:split]:
            cls = self.classes[df['class'][i]]
            order = self.orders[df['order'][i]]
            fam = self.families[df['family'][i]]
            gen = self.genuses[df['genus'][i]]
            spe = df['species_glc_id'][i]
            self.paths.append(("patchTrain/"+str(df['patch_dirname'][i])+"/patch_"+str(df['patch_id'][i])+".tif",
                               "Data/Hierarchial Data/train/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen)+"/"+str(spe)))
        
        for i in l[split:]:
            cls = self.classes[df['class'][i]]
            order = self.orders[df['order'][i]]
            fam = self.families[df['family'][i]]
            gen = self.genuses[df['genus'][i]]
            spe = df['species_glc_id'][i]
            self.paths.append(("patchTrain/"+str(df['patch_dirname'][i])+"/patch_"+str(df['patch_id'][i])+".tif",
                               "Data/Hierarchial Data/test/"+str(cls)+"/"+str(order)+"/"+str(fam)+"/"+str(gen)+"/"+str(spe)))    
    def parallelized_copying(self,item):
        global i
        i+=1
        if(i%100 == 0): 
            print(i, item[1])
        shutil.copy(item[0],item[1])
        
    def copy_data(self):
            with Pool(10) as p:
                [p.apply_async(self.parallelized_copying, args=(x,)) for x in self.paths]
                #p.map(self.paralleized_copying,self.paths)
                p.close()
                p.join()

In [5]:
ob = data_preprocessing()

In [6]:
ob.train_test_src_dest_map(train_size=0.7)

In [7]:
ob.copy_data()

100 Data/Hierarchial Data/train/6/22/63/267/1720
100 Data/Hierarchial Data/train/6/22/63/806/841
100 Data/Hierarchial Data/train/6/7/27/151/419
100 Data/Hierarchial Data/train/6/51/44/853/884
100 Data/Hierarchial Data/train/6/22/63/528/599
100 Data/Hierarchial Data/train/6/22/63/654/3172
100 Data/Hierarchial Data/train/6/35/38/237/80
100 Data/Hierarchial Data/train/8/44/123/752/2010
100 Data/Hierarchial Data/train/6/7/27/539/1611
100 Data/Hierarchial Data/train/6/29/81/418/519
200 Data/Hierarchial Data/train/6/5/19/513/120
200 Data/Hierarchial Data/train/6/21/127/52/645
200 Data/Hierarchial Data/train/6/22/63/984/169
200 Data/Hierarchial Data/train/6/50/138/6/174
200 Data/Hierarchial Data/train/6/8/29/164/281
200 Data/Hierarchial Data/train/6/5/19/361/1786
200 Data/Hierarchial Data/train/6/22/63/984/1013
200 Data/Hierarchial Data/train/6/5/19/280/384
200 Data/Hierarchial Data/train/6/1/9/731/1714
200 Data/Hierarchial Data/train/4/4/17/665/124
300 Data/Hierarchial Data/train/6/34/135/82

1800 Data/Hierarchial Data/train/6/5/19/186/299
1700 Data/Hierarchial Data/train/6/22/63/331/1759
1800 Data/Hierarchial Data/train/6/46/110/683/64
1700 Data/Hierarchial Data/train/6/29/116/738/772
1800 Data/Hierarchial Data/train/6/5/19/280/382
1800 Data/Hierarchial Data/train/6/9/35/706/753
1800 Data/Hierarchial Data/train/6/18/2/983/69
1800 Data/Hierarchial Data/train/4/43/119/97/1267
1800 Data/Hierarchial Data/train/4/43/119/154/218
1900 Data/Hierarchial Data/train/6/51/44/969/979
1800 Data/Hierarchial Data/train/6/29/81/811/72
1900 Data/Hierarchial Data/train/6/34/157/987/1022
1800 Data/Hierarchial Data/train/6/42/16/88/239
1900 Data/Hierarchial Data/train/6/29/116/738/772
1900 Data/Hierarchial Data/train/4/4/105/658/716
1900 Data/Hierarchial Data/train/6/23/64/786/60
1900 Data/Hierarchial Data/train/6/49/137/989/1247
1900 Data/Hierarchial Data/train/6/22/63/947/949
2000 Data/Hierarchial Data/train/6/22/63/588/658
1900 Data/Hierarchial Data/train/6/29/116/420/523
1900 Data/Hierarch

3400 Data/Hierarchial Data/train/6/29/102/391/479
3400 Data/Hierarchial Data/train/6/7/27/37/196
3400 Data/Hierarchial Data/train/4/43/119/460/548
3400 Data/Hierarchial Data/train/6/7/27/876/905
3500 Data/Hierarchial Data/train/6/22/63/947/956
3500 Data/Hierarchial Data/train/6/7/27/587/1642
3500 Data/Hierarchial Data/train/4/31/84/396/1825
3400 Data/Hierarchial Data/train/6/5/19/64/2908
3500 Data/Hierarchial Data/train/6/34/62/372/1800
3500 Data/Hierarchial Data/train/4/43/119/150/1448
3600 Data/Hierarchial Data/train/6/29/81/663/724
3500 Data/Hierarchial Data/train/6/7/27/185/293
3500 Data/Hierarchial Data/train/6/22/63/806/841
3500 Data/Hierarchial Data/train/1/20/60/357/451
3600 Data/Hierarchial Data/train/4/4/76/488/566
3600 Data/Hierarchial Data/train/6/22/63/528/597
3500 Data/Hierarchial Data/train/4/4/105/660/722
3600 Data/Hierarchial Data/train/6/22/63/654/1969
3600 Data/Hierarchial Data/train/6/29/81/921/1192
3600 Data/Hierarchial Data/train/6/29/156/980/995
3700 Data/Hierarc

5100 Data/Hierarchial Data/train/6/5/19/493/1889
5200 Data/Hierarchial Data/train/6/7/27/183/291
5100 Data/Hierarchial Data/train/4/43/119/154/420
5100 Data/Hierarchial Data/train/6/7/27/90/2502
5100 Data/Hierarchial Data/train/6/29/81/720/1997
5100 Data/Hierarchial Data/train/6/34/62/372/2597
5200 Data/Hierarchial Data/train/6/23/64/786/116
5200 Data/Hierarchial Data/train/4/4/105/660/9
5100 Data/Hierarchial Data/train/6/40/108/673/734
5200 Data/Hierarchial Data/train/6/9/6/44/204
5100 Data/Hierarchial Data/train/6/35/38/237/11
5300 Data/Hierarchial Data/train/6/2/11/482/562
5200 Data/Hierarchial Data/train/6/22/63/406/1289
5200 Data/Hierarchial Data/train/6/46/110/221/137
5200 Data/Hierarchial Data/train/6/47/96/383/102
5200 Data/Hierarchial Data/train/6/34/62/372/146
5300 Data/Hierarchial Data/train/6/29/81/31/190
5300 Data/Hierarchial Data/train/4/4/7/38/2769
5300 Data/Hierarchial Data/train/6/51/44/853/82
5200 Data/Hierarchial Data/train/6/9/6/223/343
5200 Data/Hierarchial Data/tr

6800 Data/Hierarchial Data/train/8/44/18/105/87
6700 Data/Hierarchial Data/train/6/34/62/372/1800
6700 Data/Hierarchial Data/train/4/4/105/636/699
6700 Data/Hierarchial Data/train/6/25/67/407/1853
6900 Data/Hierarchial Data/train/6/22/63/653/2675
6800 Data/Hierarchial Data/train/4/4/105/660/43
6900 Data/Hierarchial Data/train/4/0/12/95/416
6900 Data/Hierarchial Data/train/6/35/90/2/2155
6800 Data/Hierarchial Data/train/4/43/119/563/625
6900 Data/Hierarchial Data/train/6/29/106/125/1431
6800 Data/Hierarchial Data/train/6/7/27/448/538
6900 Data/Hierarchial Data/train/6/5/19/361/1786
6800 Data/Hierarchial Data/train/6/6/26/440/1552
6800 Data/Hierarchial Data/train/4/43/119/460/548
6900 Data/Hierarchial Data/train/6/47/132/26/2183
7000 Data/Hierarchial Data/train/8/44/51/774/158
7000 Data/Hierarchial Data/train/6/29/116/738/79
7000 Data/Hierarchial Data/train/6/29/116/738/772
6900 Data/Hierarchial Data/train/6/22/63/528/599
7000 Data/Hierarchial Data/train/6/5/19/361/367
6900 Data/Hierarch

...

15100 Data/Hierarchial Data/train/4/43/48/843/872
14900 Data/Hierarchial Data/train/6/9/35/871/898
15000 Data/Hierarchial Data/train/6/22/63/566/95
14900 Data/Hierarchial Data/train/6/22/63/307/125
15200 Data/Hierarchial Data/train/4/4/17/750/786
14900 Data/Hierarchial Data/train/8/44/18/105/250
15100 Data/Hierarchial Data/train/6/18/34/565/628
15200 Data/Hierarchial Data/train/6/1/13/433/84
15200 Data/Hierarchial Data/train/6/9/6/114/1423
15300 Data/Hierarchial Data/train/4/43/48/187/1497
15000 Data/Hierarchial Data/train/6/29/106/667/168
15200 Data/Hierarchial Data/train/6/23/64/376/148
15100 Data/Hierarchial Data/train/4/43/119/150/133
15300 Data/Hierarchial Data/train/8/44/18/105/1407
15000 Data/Hierarchial Data/train/4/43/119/119/258
15000 Data/Hierarchial Data/train/6/5/19/680/737
15300 Data/Hierarchial Data/train/6/23/23/39/1304
15300 Data/Hierarchial Data/train/6/22/63/267/373
15200 Data/Hierarchial Data/train/6/5/19/200/1512
15400 Data/Hierarchial Data/train/6/9/124/757/1058
1

16700 Data/Hierarchial Data/test/6/47/155/688/16
16700 Data/Hierarchial Data/test/6/7/27/332/1274
16800 Data/Hierarchial Data/test/6/22/63/592/664
16900 Data/Hierarchial Data/test/6/8/29/164/281
16600 Data/Hierarchial Data/test/6/29/81/418/519
16600 Data/Hierarchial Data/test/4/43/119/712/1708
16700 Data/Hierarchial Data/test/6/35/90/583/26
17000 Data/Hierarchial Data/test/6/53/41/261/161
16900 Data/Hierarchial Data/test/6/47/132/253/1541
16800 Data/Hierarchial Data/test/6/29/81/824/134
16600 Data/Hierarchial Data/test/6/34/62/372/1800
16800 Data/Hierarchial Data/test/6/22/63/890/76
16700 Data/Hierarchial Data/test/6/5/19/200/325
16900 Data/Hierarchial Data/test/4/43/48/302/398
17000 Data/Hierarchial Data/test/6/5/19/541/609
17100 Data/Hierarchial Data/test/6/34/62/597/669
17000 Data/Hierarchial Data/test/6/29/81/921/1192
16800 Data/Hierarchial Data/test/6/5/19/195/315
16700 Data/Hierarchial Data/test/6/15/42/264/1718
16900 Data/Hierarchial Data/test/7/41/114/733/1035
16700 Data/Hierar

18500 Data/Hierarchial Data/test/6/22/63/566/2665
18400 Data/Hierarchial Data/test/6/46/129/243/353
18200 Data/Hierarchial Data/test/6/53/41/261/161
18200 Data/Hierarchial Data/test/6/9/35/830/164
18600 Data/Hierarchial Data/test/6/9/100/608/1952
18400 Data/Hierarchial Data/test/6/9/3/190/312
18300 Data/Hierarchial Data/test/6/22/63/984/1011
18400 Data/Hierarchial Data/test/6/9/122/751/788
18500 Data/Hierarchial Data/test/6/6/26/55/643
18700 Data/Hierarchial Data/test/6/22/63/566/1628
18500 Data/Hierarchial Data/test/6/25/10/297/2523
18300 Data/Hierarchial Data/test/6/9/124/757/1058
18600 Data/Hierarchial Data/test/4/43/119/421/524
18300 Data/Hierarchial Data/test/6/26/68/410/505
18500 Data/Hierarchial Data/test/6/46/110/683/64
18700 Data/Hierarchial Data/test/6/5/19/964/973
18400 Data/Hierarchial Data/test/6/29/81/31/190
18800 Data/Hierarchial Data/test/6/22/63/588/658
18500 Data/Hierarchial Data/test/4/43/119/154/221
18400 Data/Hierarchial Data/test/4/43/119/563/625
18600 Data/Hierar

20100 Data/Hierarchial Data/test/4/4/76/488/1576
20000 Data/Hierarchial Data/test/6/5/19/886/913
20100 Data/Hierarchial Data/test/6/47/132/766/1064
20000 Data/Hierarchial Data/test/6/47/155/884/3300
20300 Data/Hierarchial Data/test/6/9/35/891/1169
20000 Data/Hierarchial Data/test/6/18/34/976/1231
20000 Data/Hierarchial Data/test/6/23/64/376/148
19900 Data/Hierarchial Data/test/6/22/63/947/1218
20200 Data/Hierarchial Data/test/6/5/19/7/176
20300 Data/Hierarchial Data/test/6/18/2/825/859
20100 Data/Hierarchial Data/test/6/34/62/372/464
20400 Data/Hierarchial Data/test/4/4/17/999/1261
20200 Data/Hierarchial Data/test/6/26/68/410/96
20200 Data/Hierarchial Data/test/2/27/70/412/515
20100 Data/Hierarchial Data/test/6/46/129/382/473
20100 Data/Hierarchial Data/test/4/43/119/351/2939
20100 Data/Hierarchial Data/test/6/22/63/967/977
20000 Data/Hierarchial Data/test/6/7/27/876/905
20200 Data/Hierarchial Data/test/6/29/116/982/998
20400 Data/Hierarchial Data/test/6/47/154/968/978
20300 Data/Hiera

21700 Data/Hierarchial Data/test/6/18/34/565/627
21800 Data/Hierarchial Data/test/6/22/63/947/956
21700 Data/Hierarchial Data/test/6/5/19/886/119
21800 Data/Hierarchial Data/test/6/23/64/786/116
21600 Data/Hierarchial Data/test/6/5/19/200/322
21800 Data/Hierarchial Data/test/6/9/35/871/118
21600 Data/Hierarchial Data/test/6/29/81/595/666
21900 Data/Hierarchial Data/test/6/23/64/786/814
22000 Data/Hierarchial Data/test/6/29/81/921/1192
21800 Data/Hierarchial Data/test/6/34/62/372/8
21800 Data/Hierarchial Data/test/6/25/133/813/17
21900 Data/Hierarchial Data/test/4/43/119/743/779
21900 Data/Hierarchial Data/test/6/29/81/418/519
